# Chapter 23: Code Quality and Security Scanning

While Chapter 22 validated functional correctness through testing, this chapter addresses non-functional requirements: code maintainability, structural quality, and security posture. In modern CI/CD pipelines, security cannot be a final gate—it must be integrated continuously from the first line of code. "Shifting left" on security means catching vulnerabilities during development rather than in production, reducing remediation costs by orders of magnitude.

This chapter explores automated scanning techniques that enforce code quality standards and security policies without impeding developer velocity. From static analysis that examines source code without execution to dynamic testing of running applications, these scans form protective guardrails that ensure only secure, compliant artifacts progress toward Kubernetes deployments.

## 23.1 Linting and Static Analysis

Linting and static analysis examine source code without executing it, identifying stylistic violations, potential bugs, code smells, and maintainability issues before runtime.

### Code Linting

**ESLint (JavaScript/TypeScript):**
```yaml
# GitHub Actions - ESLint with review comments
lint:
  runs-on: ubuntu-latest
  steps:
    - uses: actions/checkout@v4
    
    - name: Setup Node
      uses: actions/setup-node@v4
      with:
        node-version: '20'
        cache: 'npm'
    
    - name: Install dependencies
      run: npm ci
    
    - name: Run ESLint
      run: npx eslint . --ext .js,.jsx,.ts,.tsx --format @microsoft/eslint-formatter-sarif --output-file eslint-results.sarif
      continue-on-error: true  # Don't fail yet, upload results first
    
    - name: Upload analysis results to GitHub
      uses: github/codeql-action/upload-sarif@v2
      if: always()
      with:
        sarif_file: eslint-results.sarif
    
    - name: Check for errors
      run: npx eslint . --ext .js,.jsx,.ts,.tsx --max-warnings=0
```

**Configuration (.eslintrc.json):**
```json
{
  "extends": [
    "eslint:recommended",
    "@typescript-eslint/recommended",
    "plugin:security/recommended"
  ],
  "plugins": ["security", "sonarjs"],
  "rules": {
    "complexity": ["error", 10],
    "max-lines-per-function": ["error", 50],
    "security/detect-object-injection": "error",
    "sonarjs/cognitive-complexity": ["error", 15]
  },
  "overrides": [
    {
      "files": ["*.test.js"],
      "rules": {
        "max-lines-per-function": "off"
      }
    }
  ]
}
```

**Python (Pylint/Black/Flake8):**
```yaml
python_lint:
  stage: validate
  image: python:3.11-slim
  script:
    - pip install pylint black flake8 bandit
    - black --check --diff .
    - flake8 --max-line-length=100 --extend-ignore=E203 .
    - pylint --output-format=colorized --fail-under=8.0 src/
    - bandit -r src/ -f json -o bandit-report.json || true
  artifacts:
    reports:
      sast: bandit-report.json
    paths:
      - bandit-report.json
```

### SonarQube Integration

SonarQube provides comprehensive code quality analysis including technical debt calculation, code coverage integration, and security hotspot detection:

```yaml
sonarqube_analysis:
  stage: quality_gate
  image: sonarsource/sonar-scanner-cli:latest
  variables:
    SONAR_USER_HOME: "${CI_PROJECT_DIR}/.sonar"
    GIT_DEPTH: "0"  # Full history for blame information
  cache:
    key: "${CI_JOB_NAME}"
    paths:
      - .sonar/cache
  script:
    - sonar-scanner
      -Dsonar.projectKey=${CI_PROJECT_PATH_SLUG}
      -Dsonar.sources=.
      -Dsonar.exclusions=**/node_modules/**,**/*.test.js,**/coverage/**
      -Dsonar.javascript.lcov.reportPaths=coverage/lcov.info
      -Dsonar.qualitygate.wait=true  # Block pipeline if quality gate fails
  only:
    - merge_requests
    - main
```

**Quality Gate Configuration:**
- Coverage on new code ≥ 80%
- Duplicated lines on new code ≤ 3%
- Maintainability rating on new code ≥ A
- Security rating on new code ≥ A
- No new blocker or critical issues

### Complexity Analysis

Cyclomatic complexity measures the number of linearly independent paths through code:

```javascript
// High complexity (bad) - Cyclomatic complexity = 4
function processPayment(type, amount, currency, user) {
  if (type === 'credit') {
    if (amount > 1000) {
      if (user.verified) {
        return processCredit(amount, currency);
      }
    }
  } else if (type === 'debit') {
    return processDebit(amount);
  }
  return false;
}

// Refactored (better) - Complexity = 1 per function
function processPayment(payment) {
  const strategy = getPaymentStrategy(payment.type);
  return strategy.process(payment);
}
```

**Enforcement in CI:**
```yaml
- name: Check complexity
  run: |
    npx jscpd --threshold 5 --reporters console,json --output ./reports/
    if [ $(jq '.statistics.total.percentage' ./reports/jscpd-report.json) -gt 5 ]; then
      echo "Code duplication exceeds 5%"
      exit 1
    fi
```

## 23.2 Dependency Scanning

Third-party dependencies introduce the majority of application vulnerabilities. Dependency scanning identifies known CVEs in libraries and frameworks.

### NPM Audit and Yarn Audit

```yaml
dependency_scan:
  stage: security
  image: node:20-alpine
  script:
    - npm ci
    - npm audit --audit-level=high --json > npm-audit.json || true
    - |
      # Fail build if critical vulnerabilities found
      if npm audit --audit-level=critical 2>&1 | grep -q "found.*vulnerabilities"; then
        echo "Critical vulnerabilities found"
        exit 1
      fi
  artifacts:
    reports:
      dependency_scanning: npm-audit.json
    paths:
      - npm-audit.json
    expire_in: 1 week
```

**Automated Remediation:**
```yaml
- name: Attempt auto-fix
  run: |
    npm audit fix --force
    git diff --exit-code || (git commit -am "fix(deps): resolve security vulnerabilities" && git push)
```

### Snyk Integration

Snyk provides comprehensive vulnerability databases and fix recommendations:

```yaml
snyk_scan:
  stage: security
  image: snyk/snyk:node
  script:
    - snyk auth $SNYK_TOKEN
    - snyk test --severity-threshold=high --json-file-output=snyk-results.json
    - snyk monitor  # Track in Snyk dashboard
  artifacts:
    reports:
      dependency_scanning: snyk-results.json
  allow_failure: false  # Hard fail on vulnerabilities
```

**Snyk in Dockerfile (Scanning base images):**
```dockerfile
# Scan during build
FROM node:20-alpine AS security-scan
RUN npm install -g snyk
COPY package*.json ./
RUN snyk test --docker node:20-alpine --file=package.json
```

### OWASP Dependency-Check

For Java applications:

```yaml
owasp_dependency_check:
  stage: security
  image: owasp/dependency-check:latest
  script:
    - /usr/share/dependency-check/bin/dependency-check.sh 
        --project "$CI_PROJECT_NAME"
        --scan .
        --format JSON
        --format HTML
        --failOnCVSS 7
        --enableExperimental
  artifacts:
    paths:
      - dependency-check-report.html
    reports:
      dependency_scanning: dependency-check-report.json
```

## 23.3 Container Image Scanning

Container images bundle OS packages, libraries, and application code, creating a large attack surface that must be validated before deployment to Kubernetes.

### Trivy (Aqua Security)

Trivy scans for OS vulnerabilities (Alpine, Debian, etc.) and application dependencies:

```yaml
trivy_scan:
  stage: security
  image: aquasec/trivy:latest
  script:
    # Build image first
    - docker build -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA .
    
    # Scan image
    - trivy image --exit-code 1 --severity HIGH,CRITICAL 
        --format template --template "@contrib/sarif.tpl" 
        -o trivy-results.sarif
        $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
    
    # Generate human-readable report
    - trivy image --severity HIGH,CRITICAL 
        --format table 
        $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
  artifacts:
    reports:
      sast: trivy-results.sarif
    paths:
      - trivy-results.sarif
  allow_failure: false
```

**Scanning Base Images Only:**
```yaml
- name: Scan base image
  run: |
    # Extract base image from Dockerfile
    BASE_IMAGE=$(grep FROM Dockerfile | head -1 | awk '{print $2}')
    trivy image --severity CRITICAL $BASE_IMAGE
```

### Clair (Harbor Integration)

When using Harbor registry (Chapter 11), Clair scans automatically on push:

```yaml
# No CI configuration needed if Harbor scanning enabled
# Harbor blocks pull of vulnerable images based on policy

# Verify scan completed in CI
verify_scan:
  script:
    - |
      curl -u admin:$HARBOR_PASSWORD \
        https://harbor.company.com/api/v2.0/projects/production/repositories/myapp/artifacts/$CI_COMMIT_SHA/scan \
        | jq '.scan_overview["application/vnd.security.vulnerability.report;version=1.1"].summary.total_fixable'
```

### Distroless and Minimal Image Scanning

Scanning minimal images requires specific approaches:

```yaml
# Scan distroless images (no shell, no package manager)
- name: Scan distroless
  uses: aquasecurity/trivy-action@master
  with:
    image-ref: 'gcr.io/project/myapp:distroless'
    format: 'sarif'
    output: 'trivy-results.sarif'
    severity: 'CRITICAL,HIGH'
    exit-code: '1'
    # Skip files that don't exist in distroless
    skip-files: '/etc/os-release'
```

## 23.4 SAST (Static Application Security Testing)

SAST analyzes source code for security vulnerabilities without executing the application, detecting issues like SQL injection, XSS, and hardcoded secrets.

### Semgrep

Semgrep uses lightweight rules to find security issues across multiple languages:

```yaml
semgrep_scan:
  stage: security
  image: returntocorp/semgrep:latest
  script:
    - semgrep ci 
        --config=auto 
        --config=p/security-audit 
        --config=p/owasp-top-ten 
        --json --output=semgrep-results.json
        --sarif --output=semgrep-results.sarif
  artifacts:
    reports:
      sast: semgrep-results.sarif
    paths:
    - semgrep-results.json
    expire_in: 1 week
  rules:
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"
    - if: $CI_COMMIT_BRANCH == "main"
```

**Custom Rules for Business Logic:**
```yaml
# .semgrep.yml
rules:
  - id: insecure-password-validation
    patterns:
      - pattern: |
          if ($X.length < 8) { ... }
    languages:
      - javascript
    message: "Password length check should use constant-time comparison to prevent timing attacks"
    severity: WARNING
    
  - id: no-raw-sql
    patterns:
      - pattern: |
          db.query($X + $Y)
      - pattern: |
          db.query(`...${...}...`)
    languages:
      - javascript
      - python
    message: "Use parameterized queries to prevent SQL injection"
    severity: ERROR
```

### CodeQL (GitHub Advanced Security)

CodeQL provides semantic code analysis, treating code as data to query for vulnerabilities:

```yaml
# .github/workflows/codeql.yml
name: "CodeQL Analysis"

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]
  schedule:
    - cron: '0 9 * * 1'  # Weekly on Monday

jobs:
  analyze:
    name: Analyze
    runs-on: ubuntu-latest
    permissions:
      actions: read
      contents: read
      security-events: write
    
    strategy:
      fail-fast: false
      matrix:
        language: [ 'javascript', 'python', 'go', 'java' ]
    
    steps:
      - name: Checkout repository
        uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for better analysis
      
      - name: Initialize CodeQL
        uses: github/codeql-action/init@v2
        with:
          languages: ${{ matrix.language }}
          queries: security-extended,security-and-quality
          config: |
            paths-ignore:
              - '**/test/**'
              - '**/tests/**'
              - '**/*.test.js'
      
      - name: Autobuild
        uses: github/codeql-action/autobuild@v2
      
      - name: Perform CodeQL Analysis
        uses: github/codeql-action/analyze@v2
        with:
          category: "/language:${{matrix.language}}"
```

**CodeQL Query Examples:**
```sql
// Custom query: Find hardcoded API keys
import javascript

from StringLiteral s
where s.getValue().regexpMatch(".*[a-zA-Z0-9]{32,}.*")  // Likely API key pattern
  and not s.getFile().getBaseName().matches("%.test.%")
select s, "Potential hardcoded secret"
```

## 23.5 DAST (Dynamic Application Security Testing)

While SAST examines source code, DAST tests running applications by simulating attacks against deployed instances. DAST catches runtime vulnerabilities like authentication bypasses, misconfigurations, and exposed endpoints that static analysis misses.

### OWASP ZAP Integration

OWASP Zed Attack Proxy (ZAP) is the standard open-source DAST tool:

```yaml
dast_scan:
  stage: security
  image: owasp/zap2docker-stable:latest
  services:
    - name: $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
      alias: target
  variables:
    ZAP_TARGET: http://target:8080
    ZAP_RULES: "-1,2,3,10010"  # Alert IDs to ignore (false positives)
  script:
    # Start application and wait
    - sleep 10  # Allow service to start
    
    # Baseline scan (passive + spider)
    - zap-baseline.py 
        -t $ZAP_TARGET 
        -r zap-report.html 
        -J zap-report.json 
        -w zap-warnings.md
        -c zap-ignore.conf  # Context file for exclusions
    
    # Full active scan (takes longer, more thorough)
    # - zap-full-scan.py -t $ZAP_TARGET -r zap-full-report.html
  artifacts:
    paths:
      - zap-report.html
      - zap-report.json
    reports:
      dast: zap-report.json
    expire_in: 1 week
  allow_failure: true  # DAST can be noisy initially; tighten after baseline
```

**ZAP Configuration (zap-ignore.conf):**
```conf
# Ignore specific alerts for specific paths
10011	IGNORE	(Cookie Without Secure Flag)	.*	.*
40012	IGNORE	(Cross Domain Script Inclusion)	.*	.*	.*	https://cdn.example.com/.*
10027	IGNORE	(Information Disclosure - Suspicious Comments)	.*	.*
```

### DAST in Staging Environments

Run DAST against realistic staging deployments before production:

```yaml
# Trigger DAST after deployment to staging
stages:
  - build
  - deploy_staging
  - dast
  - deploy_production

deploy:staging:
  stage: deploy_staging
  script:
    - helm upgrade --install myapp ./chart --namespace staging
    - kubectl rollout status deployment/myapp -n staging
    
    # Expose via ingress temporarily for DAST
    - kubectl apply -f ci/staging-ingress.yaml

dast_scan:
  stage: dast
  image: owasp/zap2docker-stable
  script:
    - zap-api-scan.py 
        -t https://staging-$CI_COMMIT_SHORT_SHA.company.com 
        -f openapi 
        -r dast-report.html
  after_script:
    # Cleanup temporary ingress
    - kubectl delete -f ci/staging-ingress.yaml || true
```

### API Security Testing

For REST/GraphQL APIs, use specialized DAST:

```yaml
api_security_scan:
  image: 42crunch/scand-agent:latest
  script:
    - |
      curl -X POST "https://api.42crunch.com/api/v1/scan" \
        -H "X-API-KEY: $CRUNCH_API_KEY" \
        -d "{
          \"openapi\": \"$(cat openapi.json | base64 -w 0)\",
          \"target\": \"https://api-staging.company.com\",
          \"severity\": \"high\"
        }"
```

## 23.6 SCA (Software Composition Analysis)

SCA extends beyond simple CVE scanning to analyze license compliance, code quality of dependencies, and transitive dependency trees. It identifies not just known vulnerabilities but also unmaintained or suspicious packages.

### Sonatype Nexus Lifecycle

Enterprise-grade SCA with policy enforcement:

```yaml
nexus_scan:
  stage: security
  image: sonatype/nexus-iq-cli:latest
  script:
    - java -jar /sonatype/nexus-iq-cli.jar 
        -i myapp 
        -s https://nexus-iq.company.com 
        -a $NEXUS_USER:$NEXUS_PASS 
        -t release 
        -r nexus-iq-report.json 
        target/myapp.jar
  artifacts:
    reports:
      dependency_scanning: nexus-iq-report.json
```

**Policy Gates:**
- Security: Block on CVSS ≥ 7
- License: Block on GPL-3.0 in proprietary code
- Age: Warn on packages not updated in 2 years
- Popularity: Block on packages with < 1000 downloads/month

### GitHub Dependency Review

Prevent introducing new vulnerabilities in PRs:

```yaml
# .github/workflows/dependency-review.yml
name: 'Dependency Review'
on: [pull_request]

permissions:
  contents: read

jobs:
  dependency-review:
    runs-on: ubuntu-latest
    steps:
      - name: 'Checkout Repository'
        uses: actions/checkout@v4
      
      - name: 'Dependency Review'
        uses: actions/dependency-review-action@v3
        with:
          fail-on-severity: moderate
          # Allow specific licenses
          allow-licenses: MIT, Apache-2.0, BSD-3-Clause
          # Deny specific licenses
          deny-licenses: GPL-2.0, GPL-3.0
          # Comment summary on PR
          comment-summary-in-pr: true
```

## 23.7 License Compliance

Open source licenses vary in their requirements—some require source code disclosure (GPL), others require attribution (MIT), and commercial use may be prohibited. Automated license scanning prevents legal exposure.

### FOSSA Integration

FOSSA provides comprehensive license scanning across package managers:

```yaml
fossa_scan:
  stage: compliance
  image: fossa/fossa-cli:latest
  script:
    - fossa analyze --fossa-api-key $FOSSA_API_KEY
    - fossa test --fossa-api-key $FOSSA_API_KEY  # Fail on policy violations
  allow_failure: false  # Strict compliance requirement
```

**.fossa.yml Configuration:**
```yaml
version: 3
server: https://app.fossa.com

project:
  id: github.com/company/myapp
  name: My Application
  url: https://github.com/company/myapp

revision:
  branch: $CI_COMMIT_BRANCH

targets:
  only:
    - type: npm
      path: ./
    - type: pip
      path: ./requirements.txt

vendoredDependencies:
  forceRescans: false
  
policies:
  # Custom policy in FOSSA dashboard
  - id: proprietary-policy
  
# Exclude development dependencies
filters:
  exclude:
    - scope: devDependencies
```

### License Classification Strategy

**Permissive (Generally Safe):**
- MIT, Apache-2.0, BSD-2/3-Clause, ISC
- Require attribution but allow proprietary use

**Copyleft (Requires Care):**
- GPL-2.0, GPL-3.0, AGPL-3.0
- Require distributing source code of derivative works
- Viral: infects entire application if linked

**Prohibited:**
- No license detected (all rights reserved)
- Commercial-only licenses without purchase
- Deprecated/incompatible licenses (WTFPL, Beerware)

**CI Enforcement:**
```bash
#!/bin/bash
# license-check.sh
FORBIDDEN=("GPL-2.0" "GPL-3.0" "AGPL-3.0")
ALLOWED=("MIT" "Apache-2.0" "BSD-3-Clause")

# Check for forbidden licenses in package.json
for license in "${FORBIDDEN[@]}"; do
  if grep -r "\"license\": \"$license\"" node_modules/*/package.json 2>/dev/null; then
    echo "ERROR: Found forbidden license $license"
    exit 1
  fi
done

echo "License check passed"
```

## 23.8 Security Gates in Pipelines

Security gates prevent vulnerable code from progressing through the pipeline. These must balance security rigor with developer velocity—blocking builds on every trivial finding creates "alert fatigue," while permissive gates allow vulnerabilities to reach production.

### Severity-Based Gates

```yaml
security_gates:
  stage: security
  parallel:
    - job: critical_vulns
      script:
        - trivy image --exit-code 1 --severity CRITICAL $IMAGE
      allow_failure: false  # Never allow critical vulnerabilities
    
    - job: high_vulns
      script:
        - trivy image --exit-code 1 --severity HIGH $IMAGE
      allow_failure: true   # Warn but don't block (address in next sprint)
    
    - job: secrets_check
      script:
        - truffleHog --regex --entropy=False .
      allow_failure: false  # Hard fail on secrets exposure
```

### Break-Glass Procedures

Emergency overrides for critical hotfixes while maintaining audit trails:

```yaml
deploy:production:
  stage: deploy
  script:
    - helm upgrade --install myapp ./chart
  rules:
    # Normal: Only if security scans passed
    - if: $CI_COMMIT_BRANCH == "main" && $SECURITY_SCAN_PASSED == "true"
    
    # Emergency: Manual override with approval
    - if: $CI_COMMIT_BRANCH == "main" && $SECURITY_OVERRIDE == "true"
      when: manual
      allow_failure: false
  
  before_script:
    - |
      if [ "$SECURITY_OVERRIDE" == "true" ]; then
        echo "WARNING: Security override enabled for commit $CI_COMMIT_SHA"
        echo "Override requested by: $GITLAB_USER_LOGIN"
        echo "Reason: $OVERRIDE_REASON"
        # Send alert to security team
        curl -X POST $SECURITY_WEBHOOK \
          -d "User $GITLAB_USER_LOGIN bypassed security gates for $CI_COMMIT_SHA: $OVERRIDE_REASON"
      fi
```

### Security Debt Tracking

Allow existing vulnerabilities (grandfathering) while preventing new ones:

```yaml
# Compare current scan with baseline from main branch
security_delta:
  script:
    # Download baseline from main branch artifacts
    - curl -o baseline.json $CI_API_V4_URL/projects/$CI_PROJECT_ID/jobs/artifacts/main/raw/scan-results.json?job=security_scan
    
    # Use custom script to compare
    - python3 scripts/compare-security-scans.py 
        --baseline baseline.json 
        --current scan-results.json 
        --fail-on-new-critical
```

**Implementation Logic:**
```python
# compare-security-scans.py
import json
import sys

def main():
    with open(sys.argv[1]) as f:
        baseline = {v['id'] for v in json.load(f)['vulnerabilities']}
    
    with open(sys.argv[2]) as f:
        current = json.load(f)['vulnerabilities']
    
    new_vulns = [v for v in current if v['id'] not in baseline]
    critical_new = [v for v in new_vulns if v['severity'] == 'CRITICAL']
    
    if critical_new:
        print(f"ERROR: {len(critical_new)} new critical vulnerabilities found:")
        for v in critical_new:
            print(f"  - {v['id']}: {v['title']}")
        sys.exit(1)
    
    print(f"INFO: {len(new_vulns)} new non-critical vulnerabilities (acceptable debt)")

if __name__ == "__main__":
    main()
```

### SBOM Generation and Attestation

Software Bill of Materials (SBOM) lists all components for supply chain transparency:

```yaml
generate_sbom:
  stage: security
  image: anchore/syft:latest
  script:
    # Generate SBOM in multiple formats
    - syft packages dir:. -o spdx-json=sbom.spdx.json
    - syft packages dir:. -o cyclonedx-json=sbom.cyclonedx.json
    - syft packages dir:. -o table=sbom.txt
    
    # Sign the SBOM
    - cosign sign-blob sbom.spdx.json --output-signature sbom.spdx.json.sig
    
    # Upload to artifact repository
    - curl -X PUT $ARTIFACTORY_URL/sboms/$CI_PROJECT_NAME/$CI_COMMIT_SHA/sbom.spdx.json -T sbom.spdx.json
  artifacts:
    paths:
      - sbom.*
    reports:
      cyclonedx: sbom.cyclonedx.json  # GitLab native support
```

**Consumption in Kubernetes:**
```yaml
apiVersion: v1
kind: Pod
metadata:
  annotations:
    sbom.company.com/reference: https://artifactory.company.com/sboms/myapp/abc123/sbom.spdx.json
    sbom.company.com/verified: "true"
```

---

## Chapter Summary and Preview

In this chapter, we explored comprehensive code quality and security scanning strategies integrated into CI pipelines. We began with linting and static analysis using ESLint, SonarQube, and complexity analysis tools to enforce maintainability standards and catch potential bugs before execution. Dependency scanning with npm audit, Snyk, and OWASP Dependency-Check identified known CVEs in third-party libraries, while SAST tools including Semgrep and CodeQL analyzed source code for injection vulnerabilities, hardcoded secrets, and insecure patterns without requiring runtime execution. We examined DAST using OWASP ZAP to test running applications for runtime vulnerabilities and misconfigurations, complementing static analysis with dynamic attack simulation. SCA extended vulnerability detection to include license compliance, code quality metrics, and transitive dependency analysis, ensuring legal and operational risks are identified alongside security issues. License compliance scanning enforced organizational policies regarding copyleft and commercial restrictions, preventing intellectual property contamination. Finally, we established security gates that balance protection with velocity—blocking critical vulnerabilities immediately while tracking security debt through baseline comparisons, alongside SBOM generation for supply chain transparency and attestation.

**Key Takeaways:**
- Implement the "shift left" security principle by running lightweight scans (linting, SAST) on every commit and heavier scans (DAST, full dependency analysis) before deployment to staging
- Never allow critical severity vulnerabilities (CVSS ≥ 9.0) or exposed secrets to reach production; configure pipelines to hard fail on these findings with no override capability
- Use license scanning to prevent GPL-3.0 or AGPL contamination in proprietary software, which could force source code disclosure obligations
- Generate and sign SBOMs for every release to enable rapid vulnerability assessment when new CVEs are disclosed, providing supply chain transparency
- Establish security debt baselines that allow existing projects to grandfather non-critical findings while strictly preventing introduction of new vulnerabilities, avoiding overwhelming legacy codebases with blocking alerts

**Next Chapter Preview:**
Chapter 24: Pipeline Orchestration addresses how to structure complex CI workflows involving multiple stages, parallel execution, dependencies between pipelines, and conditional logic. We will explore advanced patterns for matrix builds across multiple environments and language versions, reusable pipeline templates that standardize security scanning across repositories, and governance mechanisms that ensure organizational compliance. This chapter examines how to scale CI from simple linear scripts to sophisticated orchestration systems that manage microservice dependencies, coordinate cross-repository changes, and integrate with enterprise approval workflows, completing the journey from individual code validation to comprehensive pipeline architecture.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='22. testing_in_ci.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='24. ci_pipeline_orchestration.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
